In [1]:
import pandas as pd
import numpy as np
import os
import shutil

def mean_crossing_rate(series):
    centered = series - series.mean()
    change = np.sign(centered).diff().abs()
    mcr = change.sum() / (2 * (len(series) - 1))
    return mcr

alpha = 0.01

for patient in np.arange(1,16):
    data = pd.read_csv(f"HAR/ppg+dalia/data/PPG_FieldStudy/S{patient}/S{patient}_E4/BVP_with_specific_activity.csv")
    
    intervals = [4,5,6,7,8] # n second intervals
    offsets = [1,2,3,4,5]
    
    for interval in intervals:
        for offset in offsets:
            window_size = 64 * interval # Number of datapoints in each interval
            shift_size = 64 * offset
            
            features = pd.DataFrame() #Collected Window Features
            
            i = 0
            while i <= len(data) - window_size: # Until Last possible starting 7 second interval
                activityLabel = 0
                window = data.loc[i:i + window_size]
                activitySubset = data.loc[i:i + window_size, "activity"]
                
                if activitySubset.nunique() != 1: # Checks if all data in window is either activity or no activity
                    actNums = (activitySubset == activitySubset.iloc[0]).sum()
                    noActNums = (activitySubset == activitySubset.iloc[-1]).sum()
                    if actNums > noActNums and noActNums / len(activitySubset) < alpha:
                        activityLabel = activitySubset.iloc[0]
                    elif actNums < noActNums and actNums / len(activitySubset) > alpha:
                        activityLabel = activitySubset.iloc[-1]
                else:
                    activityLabel = window["activity"].iloc[0]
                
                window = data.loc[i:i + window_size]
                windowFeatures = {} # Row of new features
                
                sdt = window["datetime"].iloc[0] # window start time
                windowFeatures["start_time"] = sdt
                edt = window["datetime"].iloc[-1] # window end time
                windowFeatures["end_time"] = edt
                windowFeatures["activity"] = activityLabel
                for column in window.drop(columns=['datetime', 'activity']).columns: #Loops over all three acc axes; collects features for each column
                    
                    windowFeatures[f'{column}_mean'] = window[column].mean()
                    windowFeatures[f'{column}_median'] = window[column].median()
                    windowFeatures[f'{column}_std'] = window[column].std()
                    windowFeatures[f'{column}_meancross'] = mean_crossing_rate(window[column])
                    
                    fft_result = np.fft.fft(window[column])
                    n = len(fft_result)
                    magnitude_spectrum = np.abs(fft_result)
                    freq_bins = np.fft.fftfreq(n, d=1/32)
                    
                    windowFeatures['fft_{column}_second_max'] = np.sort(magnitude_spectrum)[-2]
                    windowFeatures['fft_{column}_dc_component'] = magnitude_spectrum[0]
                    windowFeatures['fft_{column}_dominant_freq'] = freq_bins[np.argmax(magnitude_spectrum)]
                    
            
                features = pd.concat([features, pd.DataFrame([windowFeatures])], axis=0, ignore_index=True)
                
                
                i += shift_size
            
            specific_offset = f"HAR/ppg+dalia/data/PPG_FieldStudy/S{patient}/S{patient}_E4/complex_windows/length_{interval}/BVP_offset_{offset}.csv"
            
            if os.path.exists(specific_offset):
                os.remove(specific_offset)
             
            features.to_csv(specific_offset, index=False)
            print(f"S{patient} Length {interval} Offset {offset} saved!")

S1 Length 4 Offset 1 saved!
S1 Length 4 Offset 2 saved!
S1 Length 4 Offset 3 saved!
S1 Length 4 Offset 4 saved!
S1 Length 4 Offset 5 saved!
S1 Length 5 Offset 1 saved!
S1 Length 5 Offset 2 saved!
S1 Length 5 Offset 3 saved!
S1 Length 5 Offset 4 saved!
S1 Length 5 Offset 5 saved!
S1 Length 6 Offset 1 saved!
S1 Length 6 Offset 2 saved!
S1 Length 6 Offset 3 saved!
S1 Length 6 Offset 4 saved!
S1 Length 6 Offset 5 saved!
S1 Length 7 Offset 1 saved!
S1 Length 7 Offset 2 saved!
S1 Length 7 Offset 3 saved!
S1 Length 7 Offset 4 saved!
S1 Length 7 Offset 5 saved!
S1 Length 8 Offset 1 saved!
S1 Length 8 Offset 2 saved!
S1 Length 8 Offset 3 saved!
S1 Length 8 Offset 4 saved!
S1 Length 8 Offset 5 saved!
S2 Length 4 Offset 1 saved!
S2 Length 4 Offset 2 saved!
S2 Length 4 Offset 3 saved!
S2 Length 4 Offset 4 saved!
S2 Length 4 Offset 5 saved!
S2 Length 5 Offset 1 saved!
S2 Length 5 Offset 2 saved!
S2 Length 5 Offset 3 saved!
S2 Length 5 Offset 4 saved!
S2 Length 5 Offset 5 saved!
S2 Length 6 Offset 1

S12 Length 7 Offset 2 saved!
S12 Length 7 Offset 3 saved!
S12 Length 7 Offset 4 saved!
S12 Length 7 Offset 5 saved!
S12 Length 8 Offset 1 saved!
S12 Length 8 Offset 2 saved!
S12 Length 8 Offset 3 saved!
S12 Length 8 Offset 4 saved!
S12 Length 8 Offset 5 saved!
S13 Length 4 Offset 1 saved!
S13 Length 4 Offset 2 saved!
S13 Length 4 Offset 3 saved!
S13 Length 4 Offset 4 saved!
S13 Length 4 Offset 5 saved!
S13 Length 5 Offset 1 saved!
S13 Length 5 Offset 2 saved!
S13 Length 5 Offset 3 saved!
S13 Length 5 Offset 4 saved!
S13 Length 5 Offset 5 saved!
S13 Length 6 Offset 1 saved!
S13 Length 6 Offset 2 saved!
S13 Length 6 Offset 3 saved!
S13 Length 6 Offset 4 saved!
S13 Length 6 Offset 5 saved!
S13 Length 7 Offset 1 saved!
S13 Length 7 Offset 2 saved!
S13 Length 7 Offset 3 saved!
S13 Length 7 Offset 4 saved!
S13 Length 7 Offset 5 saved!
S13 Length 8 Offset 1 saved!
S13 Length 8 Offset 2 saved!
S13 Length 8 Offset 3 saved!
S13 Length 8 Offset 4 saved!
S13 Length 8 Offset 5 saved!
S14 Length 4 O